In [19]:
#ideal 8 bits

import random
import time
from braket.circuits import Circuit
from braket.aws import AwsDevice
from braket.devices import LocalSimulator

# Initialize quantum device (local simulator for testing)
device = LocalSimulator()  # Replace with AwsDevice if running on AWS Braket

# Define basis symbols
RECTILINEAR = ["|0⟩", "|1⟩"]  # |0⟩: 0°, |1⟩: 180°
DIAGONAL = ["|+⟩", "|−⟩"]  # |+⟩: 45°, |−⟩: 135°

# BB84 simulation function
def bb84_protocol(num_bits):
    start_time = time.time()

    # Step 1: Sender (Alice) generates random bitstring and bases
    alice_bits = [random.randint(0, 1) for _ in range(num_bits)]
    alice_bases = [random.choice([RECTILINEAR, DIAGONAL]) for _ in range(num_bits)]

    # Step 2: Alice prepares qubits and sends them
    circuit = Circuit()
    for i in range(num_bits):
        if alice_bases[i] == RECTILINEAR:
            if alice_bits[i] == 1:
                circuit.x(i)
        elif alice_bases[i] == DIAGONAL:
            circuit.h(i)
            if alice_bits[i] == 1:
                circuit.x(i)

    # Step 3: Receiver (Bob) chooses random bases
    bob_bases = [random.choice([RECTILINEAR, DIAGONAL]) for _ in range(num_bits)]

    # Measure Bob's qubits in his chosen bases
    for i in range(num_bits):
        if bob_bases[i] == DIAGONAL:
            circuit.h(i)
        circuit.measure(i)

    # Execute the circuit
    result = device.run(circuit, shots=1).result()
    bob_measurements = result.measurements[0]

    # Convert measurements to bitstring
    bob_bits = [int(b) for b in bob_measurements]

    # Step 4: Compare bases and generate secret key
    matched_bases = [i for i in range(num_bits) if alice_bases[i] == bob_bases[i]]
    secret_key = [alice_bits[i] for i in matched_bases]

    # Calculate communication time
    end_time = time.time()
    communication_time = end_time - start_time

    # Display results
    print("Quantum Circuit:\n", circuit)
    print("Alice's bitstring:       ", alice_bits)
    print("Alice's bases:           ", [RECTILINEAR if b == RECTILINEAR else DIAGONAL for b in alice_bases])
    print("Bob's bases:             ", [RECTILINEAR if b == RECTILINEAR else DIAGONAL for b in bob_bases])
    print("Bob's measurements:      ", bob_bits)
    print("Matched bases positions: ", matched_bases)
    print("Secret key:              ", secret_key)
    print("Communication time (s):  ", communication_time)

    # Return results for further use
    return {
        "circuit": circuit,
        "alice_bits": alice_bits,
        "alice_bases": alice_bases,
        "bob_bases": bob_bases,
        "bob_measurements": bob_bits,
        "matched_bases": matched_bases,
        "secret_key": secret_key,
        "communication_time": communication_time,
    }

# Run the BB84 protocol simulation
results = bb84_protocol(8)


Quantum Circuit:
 T  : │  0  │  1  │  2  │
      ┌───┐ ┌───┐       
q0 : ─┤ H ├─┤ M ├───────
      └───┘ └───┘       
      ┌───┐ ┌───┐       
q1 : ─┤ H ├─┤ M ├───────
      └───┘ └───┘       
      ┌───┐ ┌───┐       
q2 : ─┤ H ├─┤ M ├───────
      └───┘ └───┘       
            ┌───┐       
q3 : ───────┤ M ├───────
            └───┘       
            ┌───┐       
q4 : ───────┤ M ├───────
            └───┘       
      ┌───┐ ┌───┐ ┌───┐ 
q5 : ─┤ H ├─┤ H ├─┤ M ├─
      └───┘ └───┘ └───┘ 
      ┌───┐ ┌───┐ ┌───┐ 
q6 : ─┤ X ├─┤ H ├─┤ M ├─
      └───┘ └───┘ └───┘ 
      ┌───┐ ┌───┐ ┌───┐ 
q7 : ─┤ X ├─┤ H ├─┤ M ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │
Alice's bitstring:        [0, 0, 0, 0, 0, 0, 1, 1]
Alice's bases:            [['|0⟩', '|1⟩'], ['|0⟩', '|1⟩'], ['|0⟩', '|1⟩'], ['|0⟩', '|1⟩'], ['|0⟩', '|1⟩'], ['|+⟩', '|−⟩'], ['|0⟩', '|1⟩'], ['|0⟩', '|1⟩']]
Bob's bases:              [['|+⟩', '|−⟩'], ['|+⟩', '|−⟩'], ['|+⟩', '|−⟩'], ['|0⟩', '|1⟩'], ['|0⟩', '|1⟩'], ['|+⟩', '|−⟩'], ['|

In [18]:
#10% bitflip noise 

import random
import time
from braket.circuits import Circuit
from braket.devices import LocalSimulator

# Initialize quantum device (local simulator for testing)
device = LocalSimulator()  # Replace with AwsDevice if running on AWS Braket

# Define basis symbols
RECTILINEAR = ["|0⟩", "|1⟩"]  # |0⟩: 0°, |1⟩: 180°
DIAGONAL = ["|+⟩", "|−⟩"]  # |+⟩: 45°, |−⟩: 135°

# Bit-flip noise probability (10%)
NOISE_PROBABILITY = 0.1

# BB84 simulation function
def bb84_protocol(num_bits):
    start_time = time.time()

    # Step 1: Sender (Alice) generates random bitstring and bases
    alice_bits = [random.randint(0, 1) for _ in range(num_bits)]
    alice_bases = [random.choice([RECTILINEAR, DIAGONAL]) for _ in range(num_bits)]

    # Step 2: Alice prepares qubits and sends them
    ideal_circuit = Circuit()
    for i in range(num_bits):
        if alice_bases[i] == RECTILINEAR:
            if alice_bits[i] == 1:
                ideal_circuit.x(i)
        elif alice_bases[i] == DIAGONAL:
            ideal_circuit.h(i)
            if alice_bits[i] == 1:
                ideal_circuit.x(i)

    # Step 3: Apply bit-flip noise (10% probability)
    noisy_circuit = ideal_circuit.copy()  # Copy of the ideal circuit to apply noise
    noisy_operations = []  # To track where noise is applied

    for i in range(num_bits):
        if random.random() < NOISE_PROBABILITY:
            print(f"Applying bit-flip (X) noise on qubit {i}")  # Print where noise is applied
            noisy_circuit.x(i)  # Apply a Pauli-X gate (bit-flip)
            noisy_operations.append(i)  # Track noisy qubits

    # Step 4: Receiver (Bob) chooses random bases
    bob_bases = [random.choice([RECTILINEAR, DIAGONAL]) for _ in range(num_bits)]

    # Measure Bob's qubits in his chosen bases for both ideal and noisy circuits
    for i in range(num_bits):
        if bob_bases[i] == DIAGONAL:
            ideal_circuit.h(i)
            noisy_circuit.h(i)
        ideal_circuit.measure(i)
        noisy_circuit.measure(i)

    # Execute the ideal circuit
    ideal_result = device.run(ideal_circuit, shots=1).result()
    ideal_measurements = ideal_result.measurements[0]

    # Execute the noisy circuit
    noisy_result = device.run(noisy_circuit, shots=1).result()
    noisy_measurements = noisy_result.measurements[0]

    # Convert measurements to bitstring
    ideal_bob_bits = [int(b) for b in ideal_measurements]
    noisy_bob_bits = [int(b) for b in noisy_measurements]

    # Step 5: Compare bases and generate secret key for both ideal and noisy scenarios
    ideal_matched_bases = [i for i in range(num_bits) if alice_bases[i] == bob_bases[i]]
    noisy_matched_bases = [i for i in range(num_bits) if alice_bases[i] == bob_bases[i]]
    
    ideal_secret_key = [alice_bits[i] for i in ideal_matched_bases]
    noisy_secret_key = [alice_bits[i] for i in noisy_matched_bases]

    # Calculate communication time
    end_time = time.time()
    communication_time = end_time - start_time

    # Display results
    print("\nIdeal Circuit (Before Noise):")
    print(ideal_circuit)
    print("\nNoisy Circuit (After Bit-Flip Noise):")
    print(noisy_circuit)

    print("\nAlice's bitstring:       ", alice_bits)
    print("Alice's bases:           ", [RECTILINEAR if b == RECTILINEAR else DIAGONAL for b in alice_bases])
    print("Bob's bases:             ", [RECTILINEAR if b == RECTILINEAR else DIAGONAL for b in bob_bases])
    print("Bob's ideal measurements:", ideal_bob_bits)
    print("Bob's noisy measurements:", noisy_bob_bits)
    print("Matched bases positions (Ideal): ", ideal_matched_bases)
    print("Matched bases positions (Noisy): ", noisy_matched_bases)
    print("Ideal secret key:        ", ideal_secret_key)
    print("Noisy secret key:        ", noisy_secret_key)
    print("Communication time (s):  ", communication_time)

    # Return results for further use
    return {
        "ideal_circuit": ideal_circuit,
        "noisy_circuit": noisy_circuit,
        "alice_bits": alice_bits,
        "alice_bases": alice_bases,
        "bob_bases": bob_bases,
        "ideal_bob_measurements": ideal_bob_bits,
        "noisy_bob_measurements": noisy_bob_bits,
        "ideal_matched_bases": ideal_matched_bases,
        "noisy_matched_bases": noisy_matched_bases,
        "ideal_secret_key": ideal_secret_key,
        "noisy_secret_key": noisy_secret_key,
        "communication_time": communication_time,
        "noisy_operations": noisy_operations,
    }

# Run the BB84 protocol simulation
results = bb84_protocol(8)


Applying bit-flip (X) noise on qubit 4
Applying bit-flip (X) noise on qubit 6

Ideal Circuit (Before Noise):
T  : │  0  │  1  │  2  │  3  │
      ┌───┐       ┌───┐       
q0 : ─┤ H ├───────┤ M ├───────
      └───┘       └───┘       
                  ┌───┐       
q1 : ─────────────┤ M ├───────
                  └───┘       
      ┌───┐ ┌───┐ ┌───┐       
q2 : ─┤ X ├─┤ H ├─┤ M ├───────
      └───┘ └───┘ └───┘       
      ┌───┐ ┌───┐ ┌───┐       
q3 : ─┤ H ├─┤ X ├─┤ M ├───────
      └───┘ └───┘ └───┘       
      ┌───┐ ┌───┐ ┌───┐ ┌───┐ 
q4 : ─┤ H ├─┤ X ├─┤ H ├─┤ M ├─
      └───┘ └───┘ └───┘ └───┘ 
      ┌───┐             ┌───┐ 
q5 : ─┤ H ├─────────────┤ M ├─
      └───┘             └───┘ 
                        ┌───┐ 
q6 : ───────────────────┤ M ├─
                        └───┘ 
      ┌───┐ ┌───┐ ┌───┐ ┌───┐ 
q7 : ─┤ H ├─┤ X ├─┤ H ├─┤ M ├─
      └───┘ └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │  3  │

Noisy Circuit (After Bit-Flip Noise):
T  : │  0  │  1  │  2  │  3  │  4  │
      ┌──